# Staff Survey Analysis

In [10]:
import pandas as pd
from collections import Counter
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# Read the CSV file
df = pd.read_csv('staff_survey.csv')

# Remove metadata columns (first 6 columns)
metadata_cols = ['Start Date', 'End Date', 'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date']
survey_df = df.drop(columns=metadata_cols)

# Total number of possible responses (all rows)
total_possible = len(survey_df)

# Store results for sorting
importance_questions = []
ranking_questions = []
open_ended_questions = []

# Define the order for importance levels
importance_order = ['Extremely important', 'Very important', 'Moderately Important', 
                   'Moderately important', 'Slightly important', 'Not Important', 'Not at all important']

print(f"Analyzing survey with {total_possible} total respondents...")

# Analyze each question
for col in survey_df.columns:
    responses = survey_df[col]
    responses_not_null = responses.dropna()
    null_count = total_possible - len(responses_not_null)
    
    if len(responses_not_null) == 0:
        continue
    
    counts = Counter(responses_not_null)
    
    # Check if it's an importance question
    if any(imp in counts for imp in importance_order):
        result = {
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        }
        
        # Add counts and percentages for each importance level
        for level in importance_order:
            count = counts.get(level, 0)
            pct = (count / len(responses_not_null)) * 100 if len(responses_not_null) > 0 else 0
            result[f'{level}'] = f"{count} ({pct:.1f}%)"
        
        importance_questions.append(result)
    
    # Check if it's a ranking question (1-7)
    elif all(isinstance(r, (int, float)) for r in responses_not_null if pd.notna(r)):
        result = {
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        }
        
        # Add counts and percentages for each rank (1-7)
        for rank in range(1, 8):
            count = counts.get(rank, 0)
            pct = (count / len(responses_not_null)) * 100 if len(responses_not_null) > 0 else 0
            result[f'Rank {rank}'] = f"{count} ({pct:.1f}%)"
        
        ranking_questions.append(result)
    
    # Open-ended questions
    else:
        open_ended_questions.append({
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        })

# Create DataFrames
importance_df = pd.DataFrame(importance_questions) if importance_questions else None
ranking_df = pd.DataFrame(ranking_questions) if ranking_questions else None
open_ended_df = pd.DataFrame(open_ended_questions) if open_ended_questions else None

# Sort by response rate and top category
if importance_df is not None:
    importance_df = importance_df.sort_values('Extremely important', 
                                             key=lambda x: x.str.extract(r'(\d+)')[0].astype(int), 
                                             ascending=False)

if ranking_df is not None:
    ranking_df = ranking_df.sort_values('Rank 1', 
                                       key=lambda x: x.str.extract(r'(\d+)')[0].astype(int), 
                                       ascending=False)

if open_ended_df is not None:
    open_ended_df = open_ended_df.sort_values('Response Rate %', ascending=False)

# Function to format Excel sheet
def format_sheet(ws, df, title):
    # Set title
    ws['A1'] = title
    ws['A1'].font = Font(size=14, bold=True)
    ws['A1'].fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    ws['A1'].font = Font(size=14, bold=True, color="FFFFFF")
    ws.merge_cells(f'A1:{get_column_letter(len(df.columns))}1')
    
    # Write headers in row 2
    for col_num, column_title in enumerate(df.columns, 1):
        cell = ws.cell(row=2, column=col_num)
        cell.value = column_title
        cell.font = Font(bold=True, size=11)
        cell.fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    # Write data starting from row 3
    for r_idx, row in enumerate(df.values, 3):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx)
            cell.value = value
            
            # Align differently based on column
            if c_idx == 1:  # Question column
                cell.alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
            else:
                cell.alignment = Alignment(horizontal='center', vertical='center')
            
            # Add borders
            thin_border = Border(
                left=Side(style='thin'),
                right=Side(style='thin'),
                top=Side(style='thin'),
                bottom=Side(style='thin')
            )
            cell.border = thin_border
            
            # Alternate row colors
            if r_idx % 2 == 0:
                cell.fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
    
    # Adjust column widths
    ws.column_dimensions['A'].width = 60  # Question column
    for col_num in range(2, len(df.columns) + 1):
        ws.column_dimensions[get_column_letter(col_num)].width = 15
    
    # Freeze panes
    ws.freeze_panes = 'B3'

# Export to Excel with formatting
print("\nCreating formatted Excel file...")
with pd.ExcelWriter('survey_analysis_summary.xlsx', engine='openpyxl') as writer:
    
    if importance_df is not None:
        importance_df.to_excel(writer, sheet_name='Importance Questions', index=False, startrow=1)
        ws = writer.sheets['Importance Questions']
        format_sheet(ws, importance_df, f'IMPORTANCE QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(importance_df)} importance questions")
    
    if ranking_df is not None:
        ranking_df.to_excel(writer, sheet_name='Ranking Questions', index=False, startrow=1)
        ws = writer.sheets['Ranking Questions']
        format_sheet(ws, ranking_df, f'RANKING QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(ranking_df)} ranking questions")
    
    if open_ended_df is not None:
        open_ended_df.to_excel(writer, sheet_name='Open-Ended Questions', index=False, startrow=1)
        ws = writer.sheets['Open-Ended Questions']
        format_sheet(ws, open_ended_df, f'OPEN-ENDED QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(open_ended_df)} open-ended questions")

print("\n" + "="*80)
print("✓ ANALYSIS COMPLETE - Saved to 'survey_analysis_summary.xlsx'")
print("="*80)
print(f"\nSummary:")
print(f"  Total respondents: {total_possible}")
print(f"  Importance questions: {len(importance_questions) if importance_questions else 0}")
print(f"  Ranking questions: {len(ranking_questions) if ranking_questions else 0}")
print(f"  Open-ended questions: {len(open_ended_questions) if open_ended_questions else 0}")
print("\nOpen the Excel file to see the beautifully formatted results! 📊")

Analyzing survey with 20 total respondents...

Creating formatted Excel file...
✓ Added 78 importance questions
✓ Added 11 ranking questions
✓ Added 5 open-ended questions

✓ ANALYSIS COMPLETE - Saved to 'survey_analysis_summary.xlsx'

Summary:
  Total respondents: 20
  Importance questions: 78
  Ranking questions: 11
  Open-ended questions: 5

Open the Excel file to see the beautifully formatted results! 📊


# Faculty Survey Analysis

In [11]:
import pandas as pd
from collections import Counter
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# Read the CSV file
df = pd.read_csv('faculty_survey.csv')

# Remove metadata columns (first 6 columns)
metadata_cols = ['Start Date', 'End Date', 'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date']
survey_df = df.drop(columns=metadata_cols)

# Total number of possible responses (all rows)
total_possible = len(survey_df)

# Store results for sorting
importance_questions = []
ranking_questions = []
open_ended_questions = []

# Define the order for importance levels
importance_order = ['Extremely important', 'Very important', 'Moderately Important', 
                   'Moderately important', 'Slightly important', 'Not Important', 'Not at all important']

print(f"Analyzing faculty survey with {total_possible} total respondents...")

# Analyze each question
for col in survey_df.columns:
    responses = survey_df[col]
    responses_not_null = responses.dropna()
    null_count = total_possible - len(responses_not_null)
    
    if len(responses_not_null) == 0:
        continue
    
    counts = Counter(responses_not_null)
    
    # Check if it's an importance question
    if any(imp in counts for imp in importance_order):
        result = {
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        }
        
        # Add counts and percentages for each importance level
        for level in importance_order:
            count = counts.get(level, 0)
            pct = (count / len(responses_not_null)) * 100 if len(responses_not_null) > 0 else 0
            result[f'{level}'] = f"{count} ({pct:.1f}%)"
        
        importance_questions.append(result)
    
    # Check if it's a ranking question (1-7)
    elif all(isinstance(r, (int, float)) for r in responses_not_null if pd.notna(r)):
        result = {
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        }
        
        # Add counts and percentages for each rank (1-7)
        for rank in range(1, 8):
            count = counts.get(rank, 0)
            pct = (count / len(responses_not_null)) * 100 if len(responses_not_null) > 0 else 0
            result[f'Rank {rank}'] = f"{count} ({pct:.1f}%)"
        
        ranking_questions.append(result)
    
    # Open-ended questions
    else:
        open_ended_questions.append({
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        })

# Create DataFrames
importance_df = pd.DataFrame(importance_questions) if importance_questions else None
ranking_df = pd.DataFrame(ranking_questions) if ranking_questions else None
open_ended_df = pd.DataFrame(open_ended_questions) if open_ended_questions else None

# Sort by response rate and top category
if importance_df is not None:
    importance_df = importance_df.sort_values('Extremely important', 
                                             key=lambda x: x.str.extract(r'(\d+)')[0].astype(int), 
                                             ascending=False)

if ranking_df is not None:
    ranking_df = ranking_df.sort_values('Rank 1', 
                                       key=lambda x: x.str.extract(r'(\d+)')[0].astype(int), 
                                       ascending=False)

if open_ended_df is not None:
    open_ended_df = open_ended_df.sort_values('Response Rate %', ascending=False)

# Function to format Excel sheet
def format_sheet(ws, df, title):
    # Set title
    ws['A1'] = title
    ws['A1'].font = Font(size=14, bold=True)
    ws['A1'].fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    ws['A1'].font = Font(size=14, bold=True, color="FFFFFF")
    ws.merge_cells(f'A1:{get_column_letter(len(df.columns))}1')
    
    # Write headers in row 2
    for col_num, column_title in enumerate(df.columns, 1):
        cell = ws.cell(row=2, column=col_num)
        cell.value = column_title
        cell.font = Font(bold=True, size=11)
        cell.fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    # Write data starting from row 3
    for r_idx, row in enumerate(df.values, 3):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx)
            cell.value = value
            
            # Align differently based on column
            if c_idx == 1:  # Question column
                cell.alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
            else:
                cell.alignment = Alignment(horizontal='center', vertical='center')
            
            # Add borders
            thin_border = Border(
                left=Side(style='thin'),
                right=Side(style='thin'),
                top=Side(style='thin'),
                bottom=Side(style='thin')
            )
            cell.border = thin_border
            
            # Alternate row colors
            if r_idx % 2 == 0:
                cell.fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
    
    # Adjust column widths
    ws.column_dimensions['A'].width = 60  # Question column
    for col_num in range(2, len(df.columns) + 1):
        ws.column_dimensions[get_column_letter(col_num)].width = 15
    
    # Freeze panes
    ws.freeze_panes = 'B3'

# Export to Excel with formatting
print("\nCreating formatted Excel file...")
with pd.ExcelWriter('faculty_survey_analysis.xlsx', engine='openpyxl') as writer:
    
    if importance_df is not None:
        importance_df.to_excel(writer, sheet_name='Importance Questions', index=False, startrow=1)
        ws = writer.sheets['Importance Questions']
        format_sheet(ws, importance_df, f'IMPORTANCE QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(importance_df)} importance questions")
    
    if ranking_df is not None:
        ranking_df.to_excel(writer, sheet_name='Ranking Questions', index=False, startrow=1)
        ws = writer.sheets['Ranking Questions']
        format_sheet(ws, ranking_df, f'RANKING QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(ranking_df)} ranking questions")
    
    if open_ended_df is not None:
        open_ended_df.to_excel(writer, sheet_name='Open-Ended Questions', index=False, startrow=1)
        ws = writer.sheets['Open-Ended Questions']
        format_sheet(ws, open_ended_df, f'OPEN-ENDED QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(open_ended_df)} open-ended questions")

print("\n" + "="*80)
print("✓ ANALYSIS COMPLETE - Saved to 'faculty_survey_analysis.xlsx'")
print("="*80)
print(f"\nSummary:")
print(f"  Total respondents: {total_possible}")
print(f"  Importance questions: {len(importance_questions) if importance_questions else 0}")
print(f"  Ranking questions: {len(ranking_questions) if ranking_questions else 0}")
print(f"  Open-ended questions: {len(open_ended_questions) if open_ended_questions else 0}")
print("\nOpen the Excel file to see the beautifully formatted results! 📊")

Analyzing faculty survey with 112 total respondents...

Creating formatted Excel file...
✓ Added 79 importance questions
✓ Added 12 ranking questions
✓ Added 11 open-ended questions

✓ ANALYSIS COMPLETE - Saved to 'faculty_survey_analysis.xlsx'

Summary:
  Total respondents: 112
  Importance questions: 79
  Ranking questions: 12
  Open-ended questions: 11

Open the Excel file to see the beautifully formatted results! 📊


# Student Survey Analysis

In [13]:
import pandas as pd
from collections import Counter
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# Read the CSV file
df = pd.read_csv('student_survey.csv')

# Dynamically identify and remove metadata columns
# Metadata columns are typically: dates, progress, duration, finished, recorded date, response ID, etc.
metadata_keywords = ['start date', 'end date', 'progress', 'duration', 'finished', 
                     'recorded date', 'response id', 'ip address', 'recipient', 
                     'external reference', 'location', 'distribution channel', 'user language']

# Find columns that contain metadata keywords (case insensitive)
metadata_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in metadata_keywords)]

print(f"Identified {len(metadata_cols)} metadata columns to remove:")
for col in metadata_cols:
    print(f"  - {col}")

# Remove metadata columns
survey_df = df.drop(columns=metadata_cols)

# Total number of possible responses (all rows)
total_possible = len(survey_df)

# Store results for sorting
importance_questions = []
ranking_questions = []
open_ended_questions = []

# Define the order for importance levels
importance_order = ['Extremely important', 'Very important', 'Moderately Important', 
                   'Moderately important', 'Slightly important', 'Not Important', 'Not at all important']

print(f"Analyzing student survey with {total_possible} total respondents...")

# Analyze each question
for col in survey_df.columns:
    responses = survey_df[col]
    responses_not_null = responses.dropna()
    null_count = total_possible - len(responses_not_null)
    
    if len(responses_not_null) == 0:
        continue
    
    counts = Counter(responses_not_null)
    
    # Check if it's an importance question
    if any(imp in counts for imp in importance_order):
        result = {
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        }
        
        # Add counts and percentages for each importance level
        for level in importance_order:
            count = counts.get(level, 0)
            pct = (count / len(responses_not_null)) * 100 if len(responses_not_null) > 0 else 0
            result[f'{level}'] = f"{count} ({pct:.1f}%)"
        
        importance_questions.append(result)
    
    # Check if it's a ranking question (1-7)
    elif all(isinstance(r, (int, float)) for r in responses_not_null if pd.notna(r)):
        result = {
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        }
        
        # Add counts and percentages for each rank (1-7)
        for rank in range(1, 8):
            count = counts.get(rank, 0)
            pct = (count / len(responses_not_null)) * 100 if len(responses_not_null) > 0 else 0
            result[f'Rank {rank}'] = f"{count} ({pct:.1f}%)"
        
        ranking_questions.append(result)
    
    # Open-ended questions
    else:
        open_ended_questions.append({
            'Question': col,
            'Total Responses': len(responses_not_null),
            'No Response': null_count,
            'Response Rate %': round((len(responses_not_null) / total_possible) * 100, 1)
        })

# Create DataFrames
importance_df = pd.DataFrame(importance_questions) if importance_questions else None
ranking_df = pd.DataFrame(ranking_questions) if ranking_questions else None
open_ended_df = pd.DataFrame(open_ended_questions) if open_ended_questions else None

# Sort by response rate and top category
if importance_df is not None:
    importance_df = importance_df.sort_values('Extremely important', 
                                             key=lambda x: x.str.extract(r'(\d+)')[0].astype(int), 
                                             ascending=False)

if ranking_df is not None:
    ranking_df = ranking_df.sort_values('Rank 1', 
                                       key=lambda x: x.str.extract(r'(\d+)')[0].astype(int), 
                                       ascending=False)

if open_ended_df is not None:
    open_ended_df = open_ended_df.sort_values('Response Rate %', ascending=False)

# Function to format Excel sheet
def format_sheet(ws, df, title):
    # Set title
    ws['A1'] = title
    ws['A1'].font = Font(size=14, bold=True)
    ws['A1'].fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    ws['A1'].font = Font(size=14, bold=True, color="FFFFFF")
    ws.merge_cells(f'A1:{get_column_letter(len(df.columns))}1')
    
    # Write headers in row 2
    for col_num, column_title in enumerate(df.columns, 1):
        cell = ws.cell(row=2, column=col_num)
        cell.value = column_title
        cell.font = Font(bold=True, size=11)
        cell.fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    # Write data starting from row 3
    for r_idx, row in enumerate(df.values, 3):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx)
            cell.value = value
            
            # Align differently based on column
            if c_idx == 1:  # Question column
                cell.alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
            else:
                cell.alignment = Alignment(horizontal='center', vertical='center')
            
            # Add borders
            thin_border = Border(
                left=Side(style='thin'),
                right=Side(style='thin'),
                top=Side(style='thin'),
                bottom=Side(style='thin')
            )
            cell.border = thin_border
            
            # Alternate row colors
            if r_idx % 2 == 0:
                cell.fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
    
    # Adjust column widths
    ws.column_dimensions['A'].width = 60  # Question column
    for col_num in range(2, len(df.columns) + 1):
        ws.column_dimensions[get_column_letter(col_num)].width = 15
    
    # Freeze panes
    ws.freeze_panes = 'B3'

# Export to Excel with formatting
print("\nCreating formatted Excel file...")
with pd.ExcelWriter('student_survey_analysis.xlsx', engine='openpyxl') as writer:
    
    if importance_df is not None:
        importance_df.to_excel(writer, sheet_name='Importance Questions', index=False, startrow=1)
        ws = writer.sheets['Importance Questions']
        format_sheet(ws, importance_df, f'IMPORTANCE QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(importance_df)} importance questions")
    
    if ranking_df is not None:
        ranking_df.to_excel(writer, sheet_name='Ranking Questions', index=False, startrow=1)
        ws = writer.sheets['Ranking Questions']
        format_sheet(ws, ranking_df, f'RANKING QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(ranking_df)} ranking questions")
    
    if open_ended_df is not None:
        open_ended_df.to_excel(writer, sheet_name='Open-Ended Questions', index=False, startrow=1)
        ws = writer.sheets['Open-Ended Questions']
        format_sheet(ws, open_ended_df, f'OPEN-ENDED QUESTIONS (n={total_possible} total respondents)')
        print(f"✓ Added {len(open_ended_df)} open-ended questions")

print("\n" + "="*80)
print("✓ ANALYSIS COMPLETE - Saved to 'student_survey_analysis.xlsx'")
print("="*80)
print(f"\nSummary:")
print(f"  Total respondents: {total_possible}")
print(f"  Importance questions: {len(importance_questions) if importance_questions else 0}")
print(f"  Ranking questions: {len(ranking_questions) if ranking_questions else 0}")
print(f"  Open-ended questions: {len(open_ended_questions) if open_ended_questions else 0}")
print("\nOpen the Excel file to see the beautifully formatted results! 📊")

Identified 3 metadata columns to remove:
  - Progress
  - Duration (in seconds)
  - Finished
Analyzing student survey with 220 total respondents...

Creating formatted Excel file...
✓ Added 79 importance questions
✓ Added 4 ranking questions
✓ Added 20 open-ended questions

✓ ANALYSIS COMPLETE - Saved to 'student_survey_analysis.xlsx'

Summary:
  Total respondents: 220
  Importance questions: 79
  Ranking questions: 4
  Open-ended questions: 20

Open the Excel file to see the beautifully formatted results! 📊
